In [7]:
from importagent import *
from mesa import Agent, Model
from mesa.time import RandomActivation
import random
%matplotlib inline
# The below is needed for both notebooks and scripts
import matplotlib.pyplot as plt
import numpy as np
from mesa.space import MultiGrid
import networkx as nx
from mesa.space import NetworkGrid
import time, enum, math
from mesa.datacollection import DataCollector
from mesa import Agent, Model
from mesa.time import RandomActivation
import random
%matplotlib inline
# The below is needed for both notebooks and scripts
import matplotlib.pyplot as plt
import numpy as np
from mesa.space import MultiGrid
import networkx as nx
from mesa.space import NetworkGrid
import time, enum, math
from mesa.datacollection import DataCollector
from model import *

In [6]:



class NetworkInfectionModel(Model):
    """A model for infection spread."""

    def __init__(self, N=10, pcontact=0.4, ptrans=0.5, avg_node_degree=3,
                 progression_period=3, progression_sd=2, recovery_rate=0.0193, recovery_days=1/0.0193,
                 recovery_sd=7, incubation_time = 6 , incubation_time_sd = 2, rate_infected = 1/6):
        #My shit here:
        self.num_nodes = N  
        self.pcontact=pcontact
        n = 100
        self.A_prob = np.random.rand(N,N)
        self.A = (self.A_prob<pcontact).astype('int')
        #----------------------------#
        #           Myshit
        #
        #----------------------------#
        
        
        
        
        #self.num_nodes = N  
        #self.pcontact=pcontact
        """All parameters"""
        prob = avg_node_degree / self.num_nodes

        self.initial_outbreak_size = 1
        self.incubation_time = incubation_time
        self.incubation_time_sd = incubation_time_sd
        self.rate_infected = rate_infected
        self.recovery_sd = recovery_sd
        self.ptrans = ptrans
        self.recovery_rate = recovery_rate
        self.recovery_days = recovery_days
        self.G = nx.Graph(self.A)
        self.grid = NetworkGrid(self.G)

        self.schedule = RandomActivation(self) #activate the step of Agents in random orders
        self.running = True
        #self.dead_agents = []
        """2 types of agents: intra-agent and out-agent"""
        
        # Create out_agents: 
        
        # Create intra-agents
        for i, node in enumerate(self.G.nodes()):
            a = CovidAgent(i+1, self)
            self.G.add_node(a)
            #add agent
            self.grid.place_agent(a, node)
            self.A_prob = np.random.rand(N+i,N+i)
            self.A = (self.A_prob<pcontact).astype('int')
            #make some agents infected at start
            infected = np.random.choice([0,1], p=[0.99,0.01])
            if infected == 1:
                a.state = State.INFECTED
                a.recovery_time = self.get_recovery_time()

        self.datacollector = DataCollector(            
            agent_reporters={"State": 'state'})

    def get_recovery_time(self):
        return int(self.random.normalvariate(self.recovery_days,self.recovery_sd))

    def step(self):
        self.datacollector.collect(self)
        self.schedule.step()

In [13]:
class State(enum.IntEnum):
    SUSCEPTIBLE = 0
    EXPOSED = 1
    INFECTED = 2

class CovidAgent(Agent):
    """This is a class of agents living inside the territory of Australia, each of them have been affected by imported agents"""
    def __init__(self, unique_id, model):
        super().__init__(unique_id,model)
        self.state = State.SUSCEPTIBLE 
        self.number_of_contacts = np.random.randint(0,50)
        self.infection_time = 0
    def step(self):
        """Take a step forwards in time in 2 condition"""
        #self.move()
        if self.state == 1: 
            self.become_infected()
        if self.state == 2:
            self.give_disease()
            self.recover_to_sus()
        self.contact()
    """
    Don't need to move now
    def move(self):
       

        possible_steps = [node for node in self.model.grid.get_neighbors(self.pos, include_center=False) if self.model.grid.is_cell_empty(node)]
        if len(possible_steps) > 0:
            new_position = self.random.choice(possible_steps)                 
            self.model.grid.move_agent(self, new_position)
    """
    
    def status(self):
        """Check infection status 1 if loop for exposed condition and 1 for infected condition"""
        if self_state == State.EXPOSED: 
            successrate = self.model.success_rate
            infect = np.random.choice([0,2], p=[successrate, 1-successrate])
            if infect == 0: #if v=0 then the transmission contact becomes ineffective 
                self.state = State.SUSCEPTIBLE
            tmuy = self.model.schedule.time-self.incubation_time
            if tmuy >= self.incubation_time: 
                self.state = State.SUSCEPTIBLE
        elif self.state == State.INFECTED:     
            drate = self.model.recovery_rate
            alive = np.random.choice([0,1], p=[drate,1-drate])
            if alive == 0:
                self.model.schedule.remove(self)            
            t = self.model.schedule.time-self.infection_time
            if t >= self.recovery_time:          
                self.state = State.SUSCEPTIBLE
    def give_disease(self):
        cellmates = self.model.grid.get_cell_list_contents([self.pos])
        if len(cellmates) > 1:
            other = self.random.choice(cellmates)
            if other.state == 0:  # If susceptible
                other.state = 1
    def recover_to_sus(self):
        """Removed corersponds to recovered or died."""
        p = self.model.recovery_rate
        state = np.random.choice([0,1,2], p=[p, 1-p]) #return to either susceptible, 
        self.state = state
    def become_infected(self): 
        p = self.model.rate_infected
        state = np.random.choice([0,2], p=[1,1-p]) #contact with exposed, u either be fine or infected
    def contact(self):
        """Find close contacts and infect"""

        neighbors_nodes = self.model.grid.get_neighbors(self.pos, include_center=False)
        exposed_neighbors = [
            agent
        for agent in self.model.grid.get_cell_list_contents(neighbors_nodes)
            if agent.state is State.EXPOSED]
        for a in exposed_neighbors:
            if self.random.random() < model.ptrans:
                a.state = State.INFECTED
                a.recovery_time = model.get_recovery_time()
            else:
                a.state = State.SUSCEPTIBLE